# Korzystam z https://pokeapi.co/

## Evolution_stage_map - nazwa pokemona i numer ewolucji

In [8]:
import requests
import pandas as pd
import time
from tqdm import tqdm # Dodatkowa biblioteka do śledzenia postępu (opcjonalna, ale przydatna)

# --- FUNKCJA PRZETWARZAJĄCA ŁAŃCUCH ---
def extract_stages(chain_link, stage, results):
    """
    Rekurencyjnie przechodzi przez strukturę 'chain' i zapisuje 
    nazwę pokemona i jego etap ewolucji.
    """
    # 1. Pobieramy nazwę aktualnego Pokemona
    pokemon_name = chain_link['species']['name']
    
    # 2. Zapisujemy wynik
    results.append({
        'name': pokemon_name.capitalize(), # Zaczynamy z dużej litery, aby pasowało do Twojego CSV
        'Evolution_Stage': stage
    })
    
    # 3. Sprawdzamy, czy Pokémon ma dalsze ewolucje
    if chain_link['evolves_to']:
        # Dla każdego możliwego dalszego kroku (np. w przypadku Eevee)
        for next_link in chain_link['evolves_to']:
            # Wywołujemy funkcję ponownie dla kolejnego etapu (stage + 1)
            extract_stages(next_link, stage + 1, results)

# --- GŁÓWNA LOGIKA POBIERANIA DANYCH ---
def fetch_all_evolution_chains(max_id=540):
    """
    Pobiera wszystkie łańcuchy ewolucyjne z PokeAPI i tworzy mapę etapów.
    """
    all_results = []
    
    # Używamy tqdm do ładnego paska postępu
    for i in tqdm(range(1, max_id + 1), desc="Pobieranie łańcuchów ewolucji"):
        url = f"https://pokeapi.co/api/v2/evolution-chain/{i}/"
        
        try:
            response = requests.get(url, timeout=15)
            # Wyrzucenie błędu dla kodów 4xx/5xx (np. 404 Not Found dla nieistniejących ID)
            response.raise_for_status() 
            
            data = response.json()
            
            # Użycie naszej funkcji do przetworzenia pobranego łańcucha
            extract_stages(data['chain'], 1, all_results)
            
            # Wprowadzenie małej pauzy, aby nie przeciążać API (dobra praktyka)
            time.sleep(0.1) 

        except requests.exceptions.HTTPError as e:
            # Wystarczy zignorować, jeśli natrafimy na ID, które nie istnieje (błąd 404)
            if e.response.status_code != 404:
                print(f"\nBłąd HTTP dla ID {i}: {e}")
            continue
        except Exception as e:
            print(f"\nInny błąd dla ID {i}: {e}")
            time.sleep(1) # Dłuższa pauza po błędzie
            continue
            
    return pd.DataFrame(all_results)

# --- WYKONANIE I ZAPIS ---

# Zbieranie danych (zajmie chwilę)
df_evolution_map = fetch_all_evolution_chains()

# Zapisanie wynikowego DataFrame do pliku CSV
output_filename = 'evolution_stage_map.csv'
df_evolution_map.to_csv(output_filename, index=False)

print("\n-------------------------------------------------")
print(f" Zapisano mapę ewolucji do pliku: {output_filename}")
print("Przykład pierwszych 10 wierszy:")
print(df_evolution_map.head(10))

Pobieranie łańcuchów ewolucji: 100%|██████████| 540/540 [02:05<00:00,  4.31it/s]


-------------------------------------------------
 Zapisano mapę ewolucji do pliku: evolution_stage_map.csv
Przykład pierwszych 10 wierszy:
         name  Evolution_Stage
0   Bulbasaur                1
1     Ivysaur                2
2    Venusaur                3
3  Charmander                1
4  Charmeleon                2
5   Charizard                3
6    Squirtle                1
7   Wartortle                2
8   Blastoise                3
9    Caterpie                1


In [11]:
# --- FUNKCJA BUDUJĄCA ŚCIEŻKI ---
def build_evolution_paths(chain_link, current_path, all_paths):
    """
    Rekurencyjnie przechodzi przez strukturę 'chain' i buduje wszystkie możliwe ścieżki ewolucji.
    """
    
    # 1. Dodajemy aktualnego Pokemona do ścieżki
    pokemon_name = chain_link['species']['name'].capitalize()
    new_path = current_path + [pokemon_name]

    # 2. Sprawdzamy, czy Pokémon ma dalsze ewolucje
    if not chain_link['evolves_to']:
        # Jeśli to koniec łańcucha, zapisujemy ukończoną ścieżkę
        all_paths.append(new_path)
    else:
        # Jeśli są dalsze ewolucje, kontynuujemy budowanie ścieżek
        for next_link in chain_link['evolves_to']:
            build_evolution_paths(next_link, new_path, all_paths)

# --- GŁÓWNA LOGIKA POBIERANIA DANYCH ---
def fetch_and_map_chains(max_id=540):
    """
    Pobiera dane z API i mapuje je na horyzontalną tabelę ścieżek.
    """
    all_raw_paths = []
    
    # Pętla przez ID łańcuchów (zakładamy do 540)
    for i in range(1, max_id + 1):
        url = f"https://pokeapi.co/api/v2/evolution-chain/{i}/"
        
        try:
            response = requests.get(url, timeout=15)
            response.raise_for_status() 
            data = response.json()
            
            # Właściwe przetwarzanie: wywołujemy funkcję dla bazowego elementu łańcucha (Etap 1)
            build_evolution_paths(data['chain'], [], all_raw_paths)
            
        except requests.exceptions.HTTPError as e:
            # Ignorujemy błąd 404 Not Found dla nieistniejących ID
            if e.response.status_code != 404:
                print(f"Błąd HTTP dla ID {i}: {e}")
            continue
        except Exception as e:
            print(f"Inny błąd dla ID {i}: {e}")
            continue
            
    # Konwersja list ścieżek na DataFrame i nazwanie kolumn
    df = pd.DataFrame(all_raw_paths)
    
    # Używamy tylko 3 kolumn (Stage 1, 2, 3), resztę ignorujemy/przycinamy
    if df.shape[1] > 3:
        df = df.iloc[:, :3] 

    df.columns = [f'Stage {i+1}' for i in range(df.shape[1])]
    
    return df

# --- WYKONANIE I ZAPIS ---

# Pamiętaj o limicie zapytań do API!
df_evolution_paths = fetch_and_map_chains()

output_filename = 'evolution_paths.csv'
df_evolution_paths.to_csv(output_filename, index=False)

print("\n-------------------------------------------------")
print(f" Zapisano ścieżki ewolucyjne do pliku: {output_filename}")
print("Przykład dla pierwszych 5 wierszy (w tym rozgałęzienia):")
print(df_evolution_paths.head())


-------------------------------------------------
 Zapisano ścieżki ewolucyjne do pliku: evolution_paths.csv
Przykład dla pierwszych 5 wierszy (w tym rozgałęzienia):
      Stage 1     Stage 2     Stage 3
0   Bulbasaur     Ivysaur    Venusaur
1  Charmander  Charmeleon   Charizard
2    Squirtle   Wartortle   Blastoise
3    Caterpie     Metapod  Butterfree
4      Weedle      Kakuna    Beedrill
